In [1]:
import math

# Get inputs

In [2]:
# with open("test_input.txt") as f: inp_test = f.readlines()
with open("input.txt") as f: inp = f.readlines()
with open("gaëtan.txt") as f: gaëtan = f.readlines()

In [3]:
# Hex -> binary
converter = {
    "0" : "0000",
    "1" : "0001",
    "2" : "0010",
    "3" : "0011",
    "4" : "0100",
    "5" : "0101",
    "6" : "0110",
    "7" : "0111",
    "8" : "1000",
    "9" : "1001",
    "A" : "1010",
    "B" : "1011",
    "C" : "1100",
    "D" : "1101",
    "E" : "1110",
    "F" : "1111"}

def get_bin(hex):
    return ''.join([converter[x] for x in hex])

get_bin('0F3')

'000011110011'

## Q1 & Q2

In [4]:
def calc_ID(vals, typeID):
    """Calculate value based on type ID (switch-case waiting room)"""
    
    # Handle single values
    if isinstance(vals, int):
        return vals
    if len(vals) == 1:
        return vals[0]
    
    if typeID == 0:
        return sum(vals)
    if typeID == 1:
        return math.prod(vals)
    if typeID == 2:
        return min(vals)
    if typeID == 3:
        return max(vals)
    if typeID == 4:
        return vals[0]
    if typeID == 5:
        return 1 if vals[0] > vals[1] else 0
    if typeID == 6:
        return 0 if vals[0] > vals[1] else 1
    if typeID == 7:
        return 1 if vals[0] == vals[1] else 0
    
    print("No Valid ID")
    

In [5]:
def read_literal(bin):
    """Loops through 5 bits at a time, dropping the front bit"""
    
    fixed = ''
    for i in range(0, len(bin)+1, 5):
        fixed += bin[i+1:i+5]
        
        # If the first bit if a loop is 0, it is the last one
        if int(bin[i]) == 0:
            break

    return int(fixed,2), i+5

In [6]:
def parse_literal(bin, loc):
    """Parses packet as literal"""
    
    literal, exit_loc = read_literal(bin[loc:])
    
    return exit_loc + loc, literal

In [7]:
def parse_operator_0(bin, loc, typeID_int):
    """Parses packet as operator of sub-packets with a total length of L"""
    
    Lbin = bin[loc:loc+15]
    L = int(Lbin,2)
    end = loc + 15 + L
    
    # Begin a new parse, giving the new start and end location
    loc, new_versions, vals = begin_parse(bin, loc+15, end)
    
    # Calculate the value given the typeID
    value = calc_ID(vals, typeID_int)

    return loc, new_versions, value

In [8]:
def parse_operator_1(bin, loc, end, typeID_int):
    """Parses packet as operator with L sub-packets"""
    
    Lbin = bin[loc:loc+11]
    L = int(Lbin,2)
    
    versions = []
    values = []
    
    loc = loc + 11
    for _ in range(L):
        # Begin a new parse, giving the new start/end location, and op1
        loc, new_versions, val = begin_parse(bin, loc, end, op1=True)
        versions.extend(new_versions)
        values.extend(val)

    # Calculate the value given the typeID
    value = calc_ID(values, typeID_int)

    return loc, versions, value
    

In [9]:
def begin_parse(bin, loc=0, end=None, op1=False):
    """Consumes binary string from left to right, starting at loc=0"""
    
    versions = []
    values = []
    
    # Set end length to entire string if not specified
    if end is None:
        end = len(bin)

    # Continue consuming until there are not enough characters left
    while loc < end - 10:
        version = bin[loc:loc+3]
        versions.append(version)
        
        typeID = bin[loc+3:loc+6]
        typeID_int = int(typeID,2)
        ID = int(bin[loc+6])
        
        if typeID_int == 4:
            # print(f"version={int(version,2)}, typeID={typeID_int}, Literal packet")
            loc, val = parse_literal(bin, loc+6)
            
        elif ID == 0:
            # print(f"version={int(version,2)}, typeID={typeID_int}, Operator packet, {ID=}")
            loc, new_versions, val = parse_operator_0(bin, loc+7, typeID_int)
            versions.extend(new_versions)

        else:
            # print(f"version={int(version,2)}, typeID={typeID_int}, Operator packet, {ID=}")
            loc, new_versions, val = parse_operator_1(bin, loc+7, end, typeID_int)
            versions.extend(new_versions)

        values.append(val)

        # parse_operator_1() handles number of loops
        if op1:
            break
    
    return loc, versions, values
    

In [10]:
#_, versions, value = begin_parse(get_bin('9C0141080250320F1802104A08'))
#_, versions, value = begin_parse(get_bin(inp[0]))
_, versions, value = begin_parse(get_bin(gaëtan[0]))

print(f"Part 1 -> Sum of Versions: {sum([int(x,2) for x in versions])}")

print(f"Part 2 -> Value: {value[0]}")

Part 1 -> Sum of Versions: 883
Part 2 -> Value: 1675215002927


In [11]:
print(f"""Value should be:  1675198555015
Currently off by:      {value[0]-1675198555015}""")

Value should be:  1675198555015
Currently off by:      16447912
